Movenet 모델 저장

In [1]:
import cv2
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model

In [18]:
hub_model = hub.load('https://tfhub.dev/google/movenet/multipose/lightning/1')
#movenet = hub_model.signatures['serving_default']

In [4]:
class CustomModule(tf.Module):

  def __init__(self):
    super(CustomModule, self).__init__()
    self.v = tf.Variable(1.)

  @tf.function
  def __call__(self, x):
    return x * self.v

  @tf.function(input_signature=[tf.TensorSpec([], tf.float32)])
  def mutate(self, new_v):
    self.v.assign(new_v)

module = CustomModule()

In [5]:
call = module.__call__.get_concrete_function(tf.TensorSpec(None, tf.float32))

In [6]:
tf.saved_model.save(module, "/tmp/module_with_signature", signatures=call)

INFO:tensorflow:Assets written to: /tmp/module_with_signature\assets


INFO:tensorflow:Assets written to: /tmp/module_with_signature\assets


In [7]:
imported = tf.saved_model.load("/tmp/module_with_signature")

In [16]:
movenet = imported.signatures["serving_default"]
movenet

<ConcreteFunction signature_wrapper(*, x) at 0x232AB0027F0>

In [74]:
model = hub_model.signatures['serving_default']
model

<ConcreteFunction signature_wrapper(*, input) at 0x26F6A1524C0>

In [115]:
hub_model

<tensorflow.python.saved_model.load.Loader._recreate_base_user_object.<locals>._UserObject at 0x270970915b0>

In [26]:
hub_model.signatures.keys()

KeysView(_SignatureMap({'serving_default': <ConcreteFunction signature_wrapper(*, input) at 0x232AD43A040>}))

In [29]:
tf.saved_model.save(hub_model, "./movenet/1/", signatures=hub_model.signatures["serving_default"])

INFO:tensorflow:Assets written to: ./movenet/1/assets


INFO:tensorflow:Assets written to: ./movenet/1/assets


In [2]:
movenet = tf.saved_model.load("./movenet/1/")

In [3]:
print(list(movenet.signatures.keys()))

['serving_default']


In [4]:
movenet = movenet.signatures['serving_default']